In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'C:\Users\ankit\Downloads\ANUSHKAdocs\Datasets\fake-news\train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label', axis = 1)

In [6]:
y = df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.1.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
voc_size = 5000

In [13]:
##One hot representation

In [14]:
messages = X.copy()

In [15]:
messages.reset_index(inplace = True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ankit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [18]:
##Dataset Preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus[10]

'russian research discov secret nazi militari base treasur hunter arctic photo'

In [20]:
from tensorflow.keras.preprocessing.text import one_hot

In [21]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[4622, 317, 348, 3581, 945, 4031, 2423, 2672, 4677, 1041],
 [4889, 617, 832, 675, 2867, 3126, 4135],
 [1113, 2764, 4571, 2564],
 [4796, 2336, 3740, 4021, 3115, 2991],
 [2817, 2867, 225, 2918, 4659, 160, 2867, 3886, 1918, 148],
 [4136,
  3428,
  3537,
  3487,
  2827,
  161,
  4681,
  2538,
  4272,
  3782,
  1582,
  3523,
  2359,
  2445,
  4135],
 [364, 3515, 3092, 3581, 1425, 1697, 4090, 355, 301, 60, 2982],
 [3308, 685, 986, 108, 4192, 4955, 161, 613, 301, 60, 2982],
 [55, 1744, 1655, 4347, 3472, 397, 1968, 1555, 161, 3758],
 [1780, 2009, 363, 1418, 1431, 1472, 504, 2947],
 [1043, 2616, 945, 1585, 2387, 1342, 4038, 4983, 553, 4027, 1242],
 [4021, 2314, 945, 397, 161, 4192],
 [2157, 3777, 4760, 2501, 2194, 1366, 2464, 285, 3182],
 [280, 1485, 560, 4298, 583, 310, 385, 301, 60, 2982],
 [744, 1246, 2834, 854, 397, 301, 60, 2982],
 [228, 1387, 2930, 2648, 165, 1527, 1006, 4869, 2210, 3553],
 [2552, 2130, 617],
 [1578, 1557, 117, 2351, 161, 4984, 1948, 4135],
 [1454, 857, 832, 136, 4064, 3

In [22]:
#Embedding Representation

In [23]:
sent_len = 20
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_len)
print(embedded_docs)

[[   0    0    0 ... 2672 4677 1041]
 [   0    0    0 ... 2867 3126 4135]
 [   0    0    0 ... 2764 4571 2564]
 ...
 [   0    0    0 ...  301   60 2982]
 [   0    0    0 ... 2252 1736  608]
 [   0    0    0 ... 1399 4726  526]]


In [24]:
len(embedded_docs)

18285

In [25]:
#creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(100))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
len(embedded_docs), y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [28]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final,test_size = 0.33, random_state = 42)



In [30]:
##model training
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs =10, batch_size = 64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 14s 1ms/sample - loss: 0.3348 - accuracy: 0.8460 - val_loss: 0.1942 - val_accuracy: 0.9193
Epoch 2/10
12250/12250 [==============================] - 6s 519us/sample - loss: 0.1359 - accuracy: 0.9443 - val_loss: 0.2000 - val_accuracy: 0.9226
Epoch 3/10
12250/12250 [==============================] - 6s 511us/sample - loss: 0.0918 - accuracy: 0.9664 - val_loss: 0.2153 - val_accuracy: 0.9143
Epoch 4/10
12250/12250 [==============================] - 6s 520us/sample - loss: 0.0643 - accuracy: 0.9774 - val_loss: 0.2786 - val_accuracy: 0.9148
Epoch 5/10
12250/12250 [==============================] - 6s 517us/sample - loss: 0.0492 - accuracy: 0.9838 - val_loss: 0.3375 - val_accuracy: 0.9167
Epoch 6/10
12250/12250 [==============================] - 6s 519us/sample - loss: 0.0276 - accuracy: 0.9918 - val_loss: 0.3449 - val_accuracy: 0.9155
Epoch 7/10
12250/12250 [============================

In [32]:
##Performance Metrics and Accuracy
y_pred = model.predict_classes(X_test)


In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(y_test, y_pred)

array([[3131,  288],
       [ 258, 2358]], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9095277547638774